In [9]:
import math

class Perceptron:
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias

    def predict(self, x):
        z = sum(w * i for w, i in zip(self.weights, x)) + self.bias
        return 1 if z >= 0 else 0

# Hardcoded weights for demonstration (could also be trained)
gates_slp = {
    "AND":  {"w": [1, 1], "b": -1.5},
    "OR":   {"w": [1, 1], "b": -0.5},
    "NAND": {"w": [-1, -1], "b": 1.5},
    "NOR":  {"w": [-1, -1], "b": 0.5}
}

inputs = [[0, 0], [0, 1], [1, 0], [1, 1]]

print("--- SLP Results ---")
for name, params in gates_slp.items():
    p = Perceptron(params["w"], params["b"])
    print(f"{name} Gate: {[p.predict(i) for i in inputs]}")

--- SLP Results ---
AND Gate: [0, 0, 0, 1]
OR Gate: [0, 1, 1, 1]
NAND Gate: [1, 1, 1, 0]
NOR Gate: [1, 0, 0, 0]


In [10]:
import random

class MLP:
    def __init__(self):
        # 2 inputs -> 2 hidden neurons -> 1 output
        self.w_hidden = [[random.uniform(-1, 1) for _ in range(2)] for _ in range(2)]
        self.b_hidden = [random.uniform(-1, 1) for _ in range(2)]
        self.w_out = [random.uniform(-1, 1) for _ in range(2)]
        self.b_out = random.uniform(-1, 1)
        self.lr = 0.5

    def sigmoid(self, x):
        return 1 / (1 + math.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def train(self, inputs, targets, epochs=10000):
        for _ in range(epochs):
            total_error = 0
            for x, y in zip(inputs, targets):
                # Forward Pass
                h_act = [self.sigmoid(sum(x[i]*self.w_hidden[j][i] for i in range(2)) + self.b_hidden[j]) for j in range(2)]
                out = self.sigmoid(sum(h_act[i]*self.w_out[i] for i in range(2)) + self.b_out)
                
                error = y - out
                total_error += error**2

                # Backpropagation
                d_out = error * self.sigmoid_derivative(out)
                d_hidden = [d_out * self.w_out[i] * self.sigmoid_derivative(h_act[i]) for i in range(2)]

                # Update Weights
                for i in range(2):
                    self.w_out[i] += self.lr * d_out * h_act[i]
                self.b_out += self.lr * d_out

                for j in range(2):
                    for i in range(2):
                        self.w_hidden[j][i] += self.lr * d_hidden[j] * x[i]
                    self.b_hidden[j] += self.lr * d_hidden[j]
        return total_error / len(inputs)

    def predict(self, x):
        h_act = [self.sigmoid(sum(x[i]*self.w_hidden[j][i] for i in range(2)) + self.b_hidden[j]) for j in range(2)]
        return self.sigmoid(sum(h_act[i]*self.w_out[i] for i in range(2)) + self.b_out)

# Training Data
xor_targets = [0, 1, 1, 0]
xnor_targets = [1, 0, 0, 1]

mlp_xor = MLP()
err_xor = mlp_xor.train(inputs, xor_targets)

mlp_xnor = MLP()
err_xnor = mlp_xnor.train(inputs, xnor_targets)

print("\n--- MLP Results ---")
print(f"XOR Final Error: {err_xor:.5f}")
print(f"XOR Predictions: {[round(mlp_xor.predict(i)) for i in inputs]}")
print(f"XNOR Final Error: {err_xnor:.5f}")
print(f"XNOR Predictions: {[round(mlp_xnor.predict(i)) for i in inputs]}")


--- MLP Results ---
XOR Final Error: 0.00030
XOR Predictions: [0, 1, 1, 0]
XNOR Final Error: 0.00028
XNOR Predictions: [1, 0, 0, 1]
